In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('../input/amexpert-2019/train.csv')
submission = pd.read_csv('../input/amexpert-2019/sample_submission_Byiv0dS.csv')
coupon_item_mapping = pd.read_csv('../input/amexpert-2019/coupon_item_mapping.csv')
test = pd.read_csv('../input/amexpert-2019/test_QyjYwdj.csv')
campaign_data = pd.read_csv('../input/amexpert-2019/campaign_data.csv')
item_data = pd.read_csv('../input/amexpert-2019/item_data.csv')
customer_transaction_data = pd.read_csv('../input/amexpert-2019/customer_transaction_data.csv')
customer_demographics = pd.read_csv('../input/amexpert-2019/customer_demographics.csv')

In [ ]:
print('train :',train.shape)
print('submission :',submission.shape)
print('coupon_item_mapping :',coupon_item_mapping.shape)
print('test :',test.shape)
print('campaign_data :',campaign_data.shape)
print('item_data :',item_data.shape)
print('customer_transaction_data :',customer_transaction_data.shape)
print('customer_demographics :',customer_demographics.shape)

# train

In [ ]:
train.head()

In [ ]:
train['redemption_status'].value_counts()/train.shape[0]*100

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(train['campaign_id'])
plt.show()

In [ ]:
train.groupby(['campaign_id']).count()

In [ ]:
print(' col1 ',train[train.columns[0]].value_counts().count())
print(' col2 ',train[train.columns[1]].value_counts().count())
print(' col3 ',train[train.columns[2]].value_counts().count())
print(' col4 ',train[train.columns[3]].value_counts().count())
print(' col5 ',train[train.columns[4]].value_counts().count())
print('shape ', train.shape[0])

# campaign_data

In [ ]:
campaign_data.head()

In [ ]:
print(campaign_data.columns[0],':', campaign_data[campaign_data.columns[0]].value_counts().count())
print(campaign_data.columns[1],':', campaign_data[campaign_data.columns[1]].value_counts().count())
print(campaign_data.columns[2],':', campaign_data[campaign_data.columns[2]].value_counts().count())
print(campaign_data.columns[3],':', campaign_data[campaign_data.columns[3]].value_counts().count())
print('shape : ',campaign_data.shape[0])

In [ ]:
kl, axes = plt.subplots(2,2,figsize=(10,10))
k=0
for i in range(2):
    for j in range(2):
        sns.countplot(campaign_data[campaign_data.columns[k]], ax=axes[i,j])
        k+=1

In [ ]:
campaign_data.start_date[0][1:4]

# coupon_item_mapping

In [ ]:
coupon_item_mapping.head()

In [ ]:
print('coupon id:',coupon_item_mapping['coupon_id'].value_counts().count())
print('coupon id:',coupon_item_mapping['item_id'].value_counts().count())
print('coupon shape:',coupon_item_mapping.shape[0])

In [ ]:
coupon_item_mapping['coupon_id'].value_counts()

# item_data

In [ ]:
item_data.head()

In [ ]:
item_data.shape

In [ ]:
item_data.category.value_counts()

# customer_transaction_data

In [ ]:
customer_transaction_data.shape

In [ ]:
customer_transaction_data.head()

In [ ]:
customer_transaction_data.coupon_discount.value_counts()

In [ ]:
customer_transaction_data.coupon_discount.value_counts().index.to_frame().plot.hist(bins=200)

In [ ]:
customer_transaction_data.quantity.value_counts().index.to_frame().plot.hist(bins=500)

In [ ]:
customer_transaction_data.other_discount.value_counts()

In [ ]:
customer_transaction_data.other_discount.value_counts().index.to_frame().plot.hist(bins=500)

In [ ]:
customer_transaction_data.selling_price.value_counts()

In [ ]:
customer_transaction_data.selling_price.value_counts().index.to_frame().plot.hist(bins=500)

In [ ]:
customer_transaction_data.date.value_counts()

In [ ]:
customer_transaction_data.customer_id.value_counts()

# customer_demographics

In [ ]:
customer_demographics.shape

In [ ]:
customer_demographics.head()

In [ ]:
customer_demographics.isnull().sum()

In [ ]:
customer_demographics.age_range.value_counts()

In [ ]:
customer_demographics.marital_status.value_counts()

In [ ]:
customer_demographics.rented.value_counts()

In [ ]:
customer_demographics.family_size.value_counts()

In [ ]:
customer_demographics.no_of_children.value_counts()

In [ ]:
customer_demographics['no_of_children'].fillna(0,inplace=True)

In [ ]:
customer_demographics[['marital_status','family_size','no_of_children']].head(10)

In [ ]:
customer_demographics['family_size'].loc[customer_demographics['family_size'] == '5+'] = '5'
customer_demographics['no_of_children'].loc[customer_demographics['no_of_children'] == '3+'] = '3'

In [ ]:
customer_demographics['marital_status'].loc[(customer_demographics['marital_status'].isna()) & 
                      ((customer_demographics['family_size'].astype(int)-customer_demographics['no_of_children'].astype(int)) > 1 )] = 'Married'

In [ ]:
customer_demographics['marital_status'].fillna('Single', inplace=True)

In [ ]:
customer_demographics.isnull().sum()

# test

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
plt.figure(figsize=(20,8))
sns.distplot(test['campaign_id'], bins=30)
sns.distplot(train['campaign_id'],bins=30)
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sns.distplot(test['coupon_id'], bins=200)
sns.distplot(train['coupon_id'], bins=200)
plt.legend(['train','test'])
# sns.distplot(coupon_item_mapping['coupon_id'], bins=200)
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sns.distplot(test['customer_id'], bins=200)
sns.distplot(train['customer_id'], bins=200)
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sns.distplot(test['id'], bins=200)
sns.distplot(train['id'], bins=200)
plt.show()

# feature engineering

## train to campaign_data

In [ ]:
print(train.shape)
train.head()

In [ ]:
campaign_data.head() 

In [ ]:

def month_year(row):
    return row[3:]

from datetime import date

def date_diff(row):
    d0 = date(int('20'+str(row.start_date[6:])), int(row.start_date[3:5]), int(row.start_date[:2]))
    d1 = date(int('20'+str(row.end_date[6:])), int(row.end_date[3:5]), int(row.end_date[:2]))
    delta = d1 - d0
    return int(delta.days)

campaign_data['date_diff'] = campaign_data.apply(date_diff,axis=1)
campaign_data['start_month_year'] = campaign_data.start_date.map(month_year)
campaign_data['end_month_year'] = campaign_data.end_date.map(month_year)
campaign_data.head()

In [ ]:
campaign_data = campaign_data.drop(['start_date','end_date'],axis=1)
campaign_data['campaign_type'] = campaign_data['campaign_type'].map({'Y':1, 'X':0})
campaign_data= pd.concat((campaign_data.drop(['start_month_year','end_month_year'],axis=1),pd.get_dummies(campaign_data[['start_month_year','end_month_year']])),axis=1)
campaign_data.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
campaign_data['date_diff'] = scaler.fit_transform(campaign_data[['date_diff']])
campaign_data.head()

In [ ]:
train = train.merge(campaign_data,on = 'campaign_id', how='left')
train.head()

In [ ]:
train.shape

 ## train to coupon_item_mapping

In [ ]:
print(coupon_item_mapping.shape)
coupon_item_mapping.head()

In [ ]:
coupon_item_mapping.item_id.value_counts()

## coupon_item_mapping to item_data

In [ ]:
print(item_data.shape)
item_data.head()

In [ ]:
coupon_item_mapping = coupon_item_mapping.merge(item_data,on='item_id', how='left')
print(coupon_item_mapping.shape)
coupon_item_mapping.head()

In [ ]:
coupon_item_mapping.item_id.value_counts()

In [ ]:
df = coupon_item_mapping.groupby('coupon_id', as_index=False)['item_id'].count()
df.head()

In [ ]:
item_data.head()

In [ ]:
print(item_data.shape)
item_data.item_id.value_counts().sum()

In [ ]:
item_data = pd.concat((item_data, pd.get_dummies(item_data.brand_type)), axis=1)
item_data.head()

In [ ]:
item_data = pd.concat((item_data, pd.get_dummies(item_data.category)), axis=1)
item_data.head()

In [ ]:
print(item_data.shape)
item_data.head()

In [ ]:
item_data = item_data.drop(['brand_type', 'category'], axis=1)
print(item_data.shape)
item_data.head()

In [ ]:
coupon_item_mapping = coupon_item_mapping.merge(item_data, on='item_id', how="left")
coupon_item_mapping.shape

In [ ]:
coupon_item_mapping.columns

In [ ]:
cols = [ 'brand_x', 'brand_type', 'category', 'brand_y']

In [ ]:
coupon_item_mapping.head()

In [ ]:
coupon_item_mapping = coupon_item_mapping.drop(cols,axis=1).groupby('coupon_id', as_index = False).agg(['count','sum']).reset_index()
coupon_item_mapping.shape

In [ ]:
# List of column names
columns = ['coupon_id']

# Iterate through the variables names
for var in coupon_item_mapping.columns.levels[0]:
    # Skip the id name
    if var != 'coupon_id':
        
        # Iterate through the stat names
        for stat in coupon_item_mapping.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('item_%s_%s' % (var, stat))

print(len(columns))

In [ ]:
coupon_item_mapping.columns = columns
coupon_item_mapping.head()

In [ ]:
drop_cols = ['coupon_id',
    'item_item_id_count',
 'item_Established_sum',
 'item_Local_sum',
 'item_Alcohol_sum',
 'item_Bakery_sum',
 'item_Dairy, Juices & Snacks_sum',
 'item_Flowers & Plants_sum',
 'item_Fuel_sum',
 'item_Garden_sum',
 'item_Grocery_sum',
 'item_Meat_sum',
 'item_Miscellaneous_sum',
 'item_Natural Products_sum',
 'item_Packaged Meat_sum',
 'item_Pharmaceutical_sum',
 'item_Prepared Food_sum',
 'item_Restauarant_sum',
 'item_Salads_sum',
 'item_Seafood_sum',
 'item_Skin & Hair Care_sum',
 'item_Travel_sum',
 'item_Vegetables (cut)_sum']
coupon_item_mapping = coupon_item_mapping[drop_cols]
coupon_item_mapping

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
x = minmax.fit_transform(coupon_item_mapping.drop('coupon_id',axis=1))
x = pd.DataFrame(x,columns = drop_cols[1:])
x['coupon_id'] = coupon_item_mapping.coupon_id
x.head()

In [ ]:
train.head()

In [ ]:
# train = train.merge(coupon_item_mapping, on='coupon_id', how='left')
train = train.merge(x, on='coupon_id', how='left')
train.shape

## train to customer_id

In [ ]:
customer_transaction_data.head()

In [ ]:
cols_ex = ['date']

In [ ]:
customer_transaction_data =customer_transaction_data.drop(cols_ex,axis=1).groupby('customer_id', as_index = False).agg(['count','sum','mean','median']).reset_index()

In [ ]:
# List of column names
columns = ['customer_id']

# Iterate through the variables names
for var in customer_transaction_data.columns.levels[0]:
    # Skip the id name
    if var != 'customer_id':
        
        # Iterate through the stat names
        for stat in customer_transaction_data.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('customer_%s_%s' % (var, stat))

print(len(columns))
columns

In [ ]:
customer_transaction_data.columns = columns

In [ ]:
useful_cols = ['customer_id',
 'customer_item_id_count',
     'customer_quantity_count',
 'customer_quantity_sum',
 'customer_quantity_mean',
 'customer_quantity_median',
 'customer_selling_price_count',
 'customer_selling_price_sum',
 'customer_selling_price_mean',
 'customer_selling_price_median',
 'customer_other_discount_count',
 'customer_other_discount_sum',
 'customer_other_discount_mean',
 'customer_other_discount_median',
 'customer_coupon_discount_count',
 'customer_coupon_discount_sum',
 'customer_coupon_discount_mean',
 'customer_coupon_discount_median'
]
customer_transaction_data = customer_transaction_data[useful_cols]
customer_transaction_data

In [ ]:
minmax2 = MinMaxScaler()
x2 = minmax2.fit_transform(customer_transaction_data.drop('customer_id',axis=1))
x2 = pd.DataFrame(x2,columns = useful_cols[1:])
x2['customer_id'] = customer_transaction_data.customer_id
x2.head()

In [ ]:
train = train.merge(x2, on='customer_id', how='left')
train.shape

## train to customer_demographic

In [ ]:
customer_demographics.head()

In [ ]:
customer_demographics['no_of_children'].fillna(0,inplace=True)
customer_demographics['family_size'].loc[customer_demographics['family_size'] == '5+'] = '5'
customer_demographics['no_of_children'].loc[customer_demographics['no_of_children'] == '3+'] = '3'

In [ ]:
customer_demographics['marital_status'].loc[(customer_demographics['marital_status'].isna()) & 
                      ((customer_demographics['family_size'].astype(int)-customer_demographics['no_of_children'].astype(int)) > 1 )] = 'Married'

In [ ]:
customer_demographics['marital_status'].fillna('Single', inplace=True)

In [ ]:
customer_demographics.head()

In [ ]:
customer_demographics =pd.concat((customer_demographics.drop('age_range',axis=1), pd.get_dummies(customer_demographics['age_range'])),axis=1)

In [ ]:
customer_demographics['marital_status'] = customer_demographics['marital_status'].map({'Married':0, 'Single':1})
customer_demographics.astype(int)

In [ ]:
customer_demographics =customer_demographics.astype(int)
customer_demographics.head()

In [ ]:
scaler2 =MinMaxScaler()
customer_demographics[['family_size','no_of_children','income_bracket']] = scaler2.fit_transform(customer_demographics[['family_size','no_of_children','income_bracket']])
customer_demographics

In [ ]:
train = train.merge(customer_demographics, on='customer_id',how='left')
train.shape

In [ ]:
test = pd.read_csv('../input/amexpert-2019/test_QyjYwdj.csv')
test = test.merge(campaign_data,on = 'campaign_id', how='left')
test = test.merge(x, on='coupon_id', how='left')
test = test.merge(x2, on='customer_id', how='left')
test = test.merge(customer_demographics, on='customer_id',how='left')
test.shape

# modeling

In [ ]:
train.head()

In [ ]:
train.isnull().sum().sort_values(ascending=False)[:20].index

In [ ]:
X = train.drop(['id','campaign_id','coupon_id','customer_id','redemption_status'], axis=1)
y = pd.DataFrame(train[['redemption_status']], columns=['redemption_status'])
X.shape, y.shape

In [ ]:
X.head()

In [ ]:
corrmat = X.corrwith(y['redemption_status'])
c1 = corrmat.sort_values()[0:20].index.tolist()
c1.extend(corrmat.sort_values()[-20:].index.tolist())
len(c1)

In [ ]:
test_set = test.drop(['id','campaign_id','coupon_id','customer_id'], axis=1)
test_set.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[c1], y,test_size=0.25, random_state=78)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier(random_state=50)

# Training set
train_sets = lgb.Dataset(X_train, label = y_train)
test_sets = lgb.Dataset(X_test, label = y_test)

In [ ]:
# Default hyperparamters
hyperparameters = model.get_params()

# Using early stopping to determine number of estimators.
del hyperparameters['n_estimators']

# Perform cross validation with early stopping
cv_results = lgb.cv(hyperparameters, train_sets, num_boost_round = 10000, nfold = 5, metrics = 'auc', 
           early_stopping_rounds = 100, verbose_eval = False, seed = 42)

# Highest score
best = cv_results['auc-mean'][-1]

# Standard deviation of best score
best_std = cv_results['auc-stdv'][-1]

print('The maximium ROC AUC in cross validation was {:.5f} with std of {:.5f}.'.format(best, best_std))
print('The ideal number of iterations was {}.'.format(len(cv_results['auc-mean'])))

In [ ]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

# Optimal number of esimators found in cv
model.n_estimators = len(cv_results['auc-mean'])

# Train and make predicions with model
model.fit(X_train, y_train)
preds = model.predict_proba(X_test)[:, 1]
baseline_auc = roc_auc_score(y_test, preds)

print('The baseline model scores {:.5f} ROC AUC on the test set.'.format(baseline_auc))

In [ ]:

import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

def objective(hyperparameters):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization.
       Writes a new line to `outfile` on every iteration"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Using early stopping to find number of trees trained
    if 'n_estimators' in hyperparameters:
        del hyperparameters['n_estimators']
    
    # Retrieve the subsample
    subsample = hyperparameters['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type and subsample to top level keys
    hyperparameters['boosting_type'] = hyperparameters['boosting_type']['boosting_type']
    hyperparameters['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])

    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(hyperparameters, train_sets, num_boost_round = 10000, nfold = 5, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 50)

    run_time = timer() - start
    
    # Extract the best score
    best_score = cv_results['auc-mean'][-1]
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = len(cv_results['auc-mean'])
    
    # Add the number of estimators to the hyperparameters
    hyperparameters['n_estimators'] = n_estimators

    # Write to the csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time, best_score])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}

In [ ]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

In [ ]:
# Create the learning rate
learning_rate = {'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2))}
learning_rate

In [ ]:
learning_rate_dist = []

# Draw 10000 samples from the learning rate domain
for _ in range(10000):
    learning_rate_dist.append(sample(learning_rate)['learning_rate'])
    
plt.figure(figsize = (8, 6))
sns.kdeplot(learning_rate_dist, color = 'red', linewidth = 2, shade = True);
plt.title('Learning Rate Distribution', size = 18); plt.xlabel('Learning Rate', size = 16); plt.ylabel('Density', size = 16);

In [ ]:
# Discrete uniform distribution
num_leaves = {'num_leaves': hp.quniform('num_leaves', 30, 150, 1)}
num_leaves_dist = []

# Sample 10000 times from the number of leaves distribution
for _ in range(10000):
    num_leaves_dist.append(sample(num_leaves)['num_leaves'])
    
# kdeplot
plt.figure(figsize = (8, 6))
sns.kdeplot(num_leaves_dist, linewidth = 2, shade = True);
plt.title('Number of Leaves Distribution', size = 18); plt.xlabel('Number of Leaves', size = 16); plt.ylabel('Density', size = 16);

In [ ]:
# boosting type domain 
boosting_type = {'boosting_type': hp.choice('boosting_type', 
                                            [{'boosting_type': 'gbdt', 'subsample': hp.uniform('subsample', 0.5, 1)}, 
                                             {'boosting_type': 'dart', 'subsample': hp.uniform('subsample', 0.5, 1)},
                                             {'boosting_type': 'goss', 'subsample': 1.0}])}

# Draw a sample
hyperparams = sample(boosting_type)
hyperparams

In [ ]:
# Retrieve the subsample if present otherwise set to 1.0
subsample = hyperparams['boosting_type'].get('subsample', 1.0)

# Extract the boosting type
hyperparams['boosting_type'] = hyperparams['boosting_type']['boosting_type']
hyperparams['subsample'] = subsample

hyperparams

In [ ]:
# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                            [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                             {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                             {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'is_unbalance': hp.choice('is_unbalance', [True, False]),
}

In [ ]:
# Sample from the full space
x = sample(space)

# Conditional logic to assign top-level keys
subsample = x['boosting_type'].get('subsample', 1.0)
x['boosting_type'] = x['boosting_type']['boosting_type']
x['subsample'] = subsample

x

In [ ]:
x = sample(space)
subsample = x['boosting_type'].get('subsample', 1.0)
x['boosting_type'] = x['boosting_type']['boosting_type']
x['subsample'] = subsample
x

In [ ]:
# Create a new file and open a connection
OUT_FILE = 'bayes_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

# Test the objective function
results = objective(sample(space))
print('The cross validation loss = {:.5f}.'.format(results['loss']))
print('The optimal number of estimators was {}.'.format(results['hyperparameters']['n_estimators']))

In [ ]:
from hyperopt import tpe

# Create the algorithm
tpe_algorithm = tpe.suggest

In [ ]:
from hyperopt import Trials

# Record results
trials = Trials()

In [ ]:
# Create a file and open a connection
OUT_FILE = 'bayes_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

In [ ]:
from hyperopt import fmin

In [ ]:
# Global variable
global  ITERATION
%time
ITERATION = 0
MAX_EVALS = 5
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

best

In [ ]:
# Sort the trials with lowest loss (highest AUC) first
trials_dict = sorted(trials.results, key = lambda x: x['loss'])
trials_dict[:1]

In [ ]:
results = pd.read_csv(OUT_FILE)


In [ ]:
import ast

def evaluate(results, name):
    """Evaluate model on test data using hyperparameters in results
       Return dataframe of hyperparameters"""
    
    new_results = results.copy()
    # String to dictionary
    new_results['hyperparameters'] = new_results['hyperparameters'].map(ast.literal_eval)
    
    # Sort with best values on top
    new_results = new_results.sort_values('score', ascending = False).reset_index(drop = True)
    
    # Print out cross validation high score
    print('The highest cross validation score from {} was {:.5f} found on iteration {}.'.format(name, new_results.loc[0, 'score'], new_results.loc[0, 'iteration']))
    
    # Use best hyperparameters to create a model
    hyperparameters = new_results.loc[0, 'hyperparameters']
    model = lgb.LGBMClassifier(**hyperparameters)
    
    # Train and make predictions
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]
    
    print('ROC AUC from {} on test data = {:.5f}.'.format(name, roc_auc_score(y_test, preds)))
    
    # Create dataframe of hyperparameters
    hyp_df = pd.DataFrame(columns = list(new_results.loc[0, 'hyperparameters'].keys()))

    # Iterate through each set of hyperparameters that were evaluated
    for i, hyp in enumerate(new_results['hyperparameters']):
        hyp_df = hyp_df.append(pd.DataFrame(hyp, index = [0]), 
                               ignore_index = True)
        
    # Put the iteration and score in the hyperparameter dataframe
    hyp_df['iteration'] = new_results['iteration']
    hyp_df['score'] = new_results['score']
    
    return hyp_df

In [ ]:
%time
bayes_results = evaluate(results, name = 'Bayesian')
bayes_results


In [ ]:
MAX_EVALS = 10

# Continue training
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

In [ ]:
import json

# Save the trial results
with open('trials.json', 'w') as f:
    f.write(json.dumps(trials_dict))

In [ ]:
bayes_results = pd.read_csv('../input/home-credit-model-tuning/bayesian_trials_1000.csv').sort_values('score', ascending = False).reset_index()
random_results = pd.read_csv('../input/home-credit-model-tuning/random_search_trials_1000.csv').sort_values('score', ascending = False).reset_index()
random_results['loss'] = 1 - random_results['score']

bayes_params = evaluate(bayes_results, name = 'Bayesian')
random_params = evaluate(random_results, name = 'random')

In [ ]:
# Dataframe of just scores
scores = pd.DataFrame({'ROC AUC': random_params['score'], 'iteration': random_params['iteration'], 'search': 'Random'})
scores = scores.append(pd.DataFrame({'ROC AUC': bayes_params['score'], 'iteration': bayes_params['iteration'], 'search': 'Bayesian'}))

scores['ROC AUC'] = scores['ROC AUC'].astype(np.float32)
scores['iteration'] = scores['iteration'].astype(np.int32)

scores.head()

In [ ]:
best_random_params = random_params.iloc[random_params['score'].idxmax(), :].copy()
best_bayes_params = bayes_params.iloc[bayes_params['score'].idxmax(), :].copy()

In [ ]:
# Plot of scores over the course of searching
sns.lmplot('iteration', 'ROC AUC', hue = 'search', data = scores, size = 8);
plt.scatter(best_bayes_params['iteration'], best_bayes_params['score'], marker = '*', s = 400, c = 'orange', edgecolor = 'k')
plt.scatter(best_random_params['iteration'], best_random_params['score'], marker = '*', s = 400, c = 'blue', edgecolor = 'k')
plt.xlabel('Iteration'); plt.ylabel('ROC AUC'); plt.title("Validation ROC AUC versus Iteration");

In [ ]:
import altair as alt

alt.renderers.enable('notebook')

c = alt.Chart(scores).mark_circle().encode(x = 'iteration', y = alt.Y('ROC AUC', 
                                                                  scale = alt.Scale(domain = [0.64, 0.74])),
                                       color = 'search')
c.title = 'Validation ROC AUC vs Iteration'
c

In [ ]:
plt.figure(figsize = (20, 8))
plt.rcParams['font.size'] = 18

# Density plots of the learning rate distributions 
sns.kdeplot(learning_rate_dist, label = 'Sampling Distribution', linewidth = 4)
sns.kdeplot(random_params['learning_rate'], label = 'Random Search', linewidth = 4)
sns.kdeplot(bayes_params['learning_rate'], label = 'Bayes Optimization', linewidth = 4)
plt.vlines([best_random_params['learning_rate'], best_bayes_params['learning_rate']],
           ymin = 0.0, ymax = 50.0, linestyles = '--', linewidth = 4, colors = ['orange', 'green'])
plt.legend()
plt.xlabel('Learning Rate'); plt.ylabel('Density'); plt.title('Learning Rate Distribution');

In [ ]:
# Iterate through each hyperparameter
for i, hyper in enumerate(random_params.columns):
    if hyper not in ['class_weight', 'n_estimators', 'score', 'is_unbalance',
                    'boosting_type', 'iteration', 'subsample', 'metric', 'verbose', 'loss', 'learning_rate']:
        plt.figure(figsize = (14, 6))
        # Plot the random search distribution and the bayes search distribution
        if hyper != 'loss':
            sns.kdeplot([sample(space[hyper]) for _ in range(1000)], label = 'Sampling Distribution', linewidth = 4)
        sns.kdeplot(random_params[hyper], label = 'Random Search', linewidth = 4)
        sns.kdeplot(bayes_params[hyper], label = 'Bayes Optimization', linewidth = 4)
        plt.vlines([best_random_params[hyper], best_bayes_params[hyper]],
                     ymin = 0.0, ymax = 10.0, linestyles = '--', linewidth = 4, colors = ['orange', 'green'])
        plt.legend(loc = 1)
        plt.title('{} Distribution'.format(hyper))
        plt.xlabel('{}'.format(hyper)); plt.ylabel('Density');
        plt.show();

In [ ]:
fig, axs = plt.subplots(1, 4, figsize = (24, 6))
i = 0

# Plot of four hyperparameters
for i, hyper in enumerate(['colsample_bytree', 'learning_rate', 'min_child_samples', 'num_leaves']):
    
        # Scatterplot
        sns.regplot('iteration', hyper, data = bayes_params, ax = axs[i])
        axs[i].scatter(best_bayes_params['iteration'], best_bayes_params[hyper], marker = '*', s = 200, c = 'k')
        axs[i].set(xlabel = 'Iteration', ylabel = '{}'.format(hyper), title = '{} over Search'.format(hyper));

plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1, 4, figsize = (24, 6))
i = 0

# Scatterplot of next three hyperparameters
for i, hyper in enumerate(['reg_alpha', 'reg_lambda', 'subsample_for_bin', 'subsample']):
        sns.regplot('iteration', hyper, data = bayes_params, ax = axs[i])
        axs[i].scatter(best_bayes_params['iteration'], best_bayes_params[hyper], marker = '*', s = 200, c = 'k')
        axs[i].set(xlabel = 'Iteration', ylabel = '{}'.format(hyper), title = '{} over Search'.format(hyper));

plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1, 2, sharey = True, sharex = True)

# Bar plots of boosting type
random_params['boosting_type'].value_counts().plot.bar(ax = axs[0], figsize = (14, 6), color = 'orange', title = 'Random Search Boosting Type')
bayes_params['boosting_type'].value_counts().plot.bar(ax = axs[1], figsize = (14, 6), color = 'green', title = 'Bayes Optimization Boosting Type');

In [ ]:
bars = alt.Chart(random_params, width = 500).mark_bar(color = 'orange').encode(x = 'boosting_type', y = alt.Y('count()', scale = alt.Scale(domain = [0, 400])))
text = bars.mark_text(size = 20, align = 'center', baseline = 'bottom').encode(text = 'count()')

bars + text

In [ ]:
bars = alt.Chart(bayes_params, width = 500).mark_bar(color = 'green').encode(x = 'boosting_type', y = alt.Y('count()', scale = alt.Scale(domain = [0, 800])))
text = bars.mark_text(size = 20, align = 'center', baseline = 'bottom').encode(text = 'count()')

bars + text

In [ ]:
random_results['hyperparameters'] = random_results['hyperparameters'].map(ast.literal_eval)
bayes_results['hyperparameters'] = bayes_results['hyperparameters'].map(ast.literal_eval)

In [ ]:
train_set2 = lgb.Dataset(X, label = y)

hyperparameters = dict(**random_results.loc[0, 'hyperparameters'])
del hyperparameters['n_estimators']

# Cross validation with n_folds and early stopping
cv_results = lgb.cv(hyperparameters, train_set2,
                    num_boost_round = 10000, early_stopping_rounds = 100, 
                    metrics = 'auc', nfold = 5)

print('The cross validation score on the full dataset  for Random Search= {:.5f} with std: {:.5f}.'.format(
    cv_results['auc-mean'][-1], cv_results['auc-stdv'][-1]))
print('Number of estimators = {}.'.format(len(cv_results['auc-mean'])))

In [ ]:
model = lgb.LGBMClassifier(n_estimators = len(cv_results['auc-mean']), **hyperparameters)
model.fit(X, y)

final_preds = model.predict_proba(test_set)[:, 1]

# submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': preds})
# submission.to_csv('submission_random_search.csv', index = False)

In [ ]:
preds = model.predict(X_test)
print(classification_report(preds, y_test))
print(f1_score(preds, y_test))
print(roc_auc_score(preds, y_test))

In [ ]:
prediction = model.predict(test_set[c1])
submission['redemption_status'] = prediction
print(submission.shape)
submission.head()

In [ ]:
y['redemption_status'].value_counts()

In [ ]:
submission['redemption_status'].value_counts()

In [ ]:
from IPython.display import HTML
import base64

def create_download_link(df, title = "Download CSV file", filename = "output.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)